In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from scipy.stats import ks_2samp

In [6]:
books_dataset = 'final_book_dataset_kaggle.csv'

In [7]:
books = pd.read_csv(books_dataset)
books.head()

,title,author,price,pages,avg_reviews,n_reviews,star5,star4,star3,star2,star1,dimensions,weight,language,publisher,ISBN_13,link,complete_link
0,Becoming a Data Head: How to Think Speak and U...,"[Alex J. Gutman,Jordan Goldmeier]",24.49,272.0,4.6,184,0.74,0.18,0.05,0.02,0.01,6 x 0.62 x 9 inches,12.5 ounces,English,Wiley; 1st edition (April 23 2021),978-1119741749,/Becoming-Data-Head-Understand-Statistics/dp/1...,https://www.amazon.com/Becoming-Data-Head-Unde...
1,Ace the Data Science Interview: 201 Real Inter...,"[Nick Singh,Kevin Huo]",26.00,301.0,4.5,599,0.77,0.10,0.06,0.03,0.04,7 x 0.68 x 10 inches,1.28 pounds,English,Ace the Data Science Interview (August 16 2021),978-0578973838,/Ace-Data-Science-Interview-Questions/dp/05789...,https://www.amazon.com/Ace-Data-Science-Interv...
2,Fundamentals of Data Engineering: Plan and Bui...,"[Joe Reis,Matt Housley]",50.76,446.0,5.0,33,0.96,0.04,0.00,0.00,0.00,7 x 1 x 9.25 inches,1.57 pounds,English,OReilly Media; 1st edition (July 26 2022),978-1098108304,/Fundamentals-Data-Engineering-Robust-Systems/...,https://www.amazon.com/Fundamentals-Data-Engin...
3,Essential Math for Data Science: Take Control ...,[Thomas Nield],44.80,347.0,4.5,27,0.79,0.05,0.05,0.05,0.05,7 x 0.75 x 9 inches,1.23 pounds,English,OReilly Media; 1st edition (July 5 2022),978-1098102937,/Essential-Math-Data-Science-Fundamental/dp/10...,https://www.amazon.com/Essential-Math-Data-Sci...
4,Data Science for Business: What You Need to Kn...,"[Foster Provost,Tom Fawcett]",36.99,413.0,4.5,970,0.71,0.15,0.08,0.03,0.03,7 x 0.9 x 9.19 inches,1.5 pounds,English,OReilly Media; 1st edition (September 17 2013),978-1449361327,/Data-Science-Business-Data-Analytic-Thinking/...,https://www.amazon.com/Data-Science-Business-D...


In [8]:
books.isnull().mean()

title            0.000000
author           0.128964
price            0.002114
pages            0.012685
avg_reviews      0.130021
n_reviews        0.000000
star5            0.000000
star4            0.000000
star3            0.000000
star2            0.000000
star1            0.000000
dimensions       0.033827
weight           0.003171
language         0.003171
publisher        0.004228
ISBN_13          0.001057
link             0.000000
complete_link    0.000000
dtype: float64

In [27]:
 def ks_test(data,missing_col,compare_col):
        null = df.copy()
        null['is_null'] = df[col_null].isnull()
        missing_data = data.loc[data[missing_col],compare_col]
        nonmissing_data = data.loc[~data[missing_col],compare_col]
        return ks_2samp(missing_data, nonmissing_data).pvalue

In [9]:
def tvd_by_col(df, col): 
    ''' calculates the given tvd based on column name, requires is_null column '''
    pivot = df.pivot_table(index='is_null', columns=col,
                           aggfunc='size', fill_value=0)
    pivot_norm = pivot.apply(lambda x: x / x.sum(), axis=1)

    tvd = pivot_norm.diff().abs().iloc[-1].sum() / 2
    return tvd

In [10]:
def missingness_ptest(df, col_null, col_sus):
    ''' 
    computes a permutation of missingness on col_null in regards to col_sus
    Returns the observed and p-value in tuple
    '''
    
    null = df.copy()
    null['is_null'] = df[col_null].isnull()
    
    obs = tvd_by_col(null, col_sus)
    
    shuffled = null.copy()
    tvds = []
    
    trials = 1000
    for _ in range(trials):
        shuffled_col = null[col_sus].sample(replace=False, frac=1).reset_index(drop=True)
        
        shuffled[col_sus] = shuffled_col
        
        tvd = tvd_by_col(shuffled, col_sus)
        tvds.append(tvd)
    
    p_value = (tvds >= obs).mean()
    
    return obs, p_value, tvds

In [11]:
ps = pd.read_csv('googleplaystore.csv')

In [12]:
ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [30]:
ps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


In [90]:
ps.isnull().mean(), ps.isnull().sum()

(App                0.000000
 Category           0.000000
 Rating             0.135965
 Reviews            0.000000
 Size               0.000000
 Installs           0.000000
 Type               0.000092
 Price              0.000000
 Content Rating     0.000092
 Genres             0.000000
 Last Updated       0.000000
 Current Ver        0.000738
 Android Ver        0.000277
 Reviews_cleaned    0.000000
 Price_claned       0.000000
 Price_cleaned      0.000000
 dtype: float64,
 App                   0
 Category              0
 Rating             1474
 Reviews               0
 Size                  0
 Installs              0
 Type                  1
 Price                 0
 Content Rating        1
 Genres                0
 Last Updated          0
 Current Ver           8
 Android Ver           3
 Reviews_cleaned       0
 Price_claned          0
 Price_cleaned         0
 dtype: int64)

In [26]:
obs, p, tvds = missingness_ptest(ps, 'Rating', 'Genres')
print('p-value: {}'.format(p))

p-value: 0.0


In [40]:
ps.Reviews.unique()

array(['159', '967', '87510', ..., '603', '1195', '398307'], dtype=object)

In [51]:
ps[ps.Reviews.str.find('M') != -1]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
10472,Life Made WI-Fi Touchscreen Photo Frame,1.9,19.0,3.0M,"1,000+",Free,0,Everyone,NaN,"February 11, 2018",1.0.19,4.0 and up,NaN


In [54]:
def assess_missingness(data,focus_col,compare_col,stat="tvd",n_repetitions=1000):
    def tvd(data,missing_col,compare_col):
        pivoted = (
            shuffled
            .pivot_table(index=missing_col, columns=compare_col, aggfunc='size')
            .apply(lambda x: x / x.sum(), axis=1)
        )

        tvd = pivoted.diff().iloc[-1].abs().sum() / 2
        return tvd
    
    def ks_test(data,missing_col,compare_col):
        missing_data = data.loc[data[missing_col],compare_col]
        nonmissing_data = data.loc[~data[missing_col],compare_col]
        return ks_2samp(missing_data, nonmissing_data).pvalue
    
    shuffled = data.copy()
    missing_col = focus_col+'_missing'
    shuffled[missing_col] = shuffled[focus_col].isna()
    
    
    if stat == "ks":
        return ks_test(shuffled,missing_col,compare_col)
    
    if stat == "tvd":
        obs_tvd = tvd(data,missing_col,compare_col)
        tvds = []
        for _ in range(n_repetitions):

            # Shuffling genders and assigning back to the DataFrame
            shuffled[compare_col] = np.random.permutation(shuffled[compare_col])
            tvds.append(tvd(shuffled,missing_col,compare_col))
        pval = np.mean(np.array(tvds) >= obs_tvd)
        return pval

In [58]:
ps['Reviews_cleaned'] = ps.Reviews.str.replace('M', '').astype(float)

In [56]:
assess_missingness(ps,"Rating","Genres")

0.0

In [60]:
assess_missingness(ps,"Rating","Reviews_cleaned", stat='ks')

1.057e-321

In [63]:
ps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Reviews_cleaned'],
      dtype='object')

In [66]:
ps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Reviews_cleaned
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,159.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,967.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,87510.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,215644.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,967.0


In [67]:
assess_missingness(ps, "Rating", "Type")

0.0

In [88]:
ps['Price_cleaned'] = ps.Price.str.replace('$', '').replace('Everyone', '0').astype(float)

/var/folders/42/7620rj0s40l61g76z97k379m0000gn/T/ipykernel_39740/2670680486.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ps['Price_cleaned'] = ps.Price.str.replace('$', '').replace('Everyone', '0').astype(float)


In [89]:
assess_missingness(ps,"Rating","Price_cleaned", stat='ks')

0.0902642494540807